<h3>The first cell setup the initialization process. It needs to be run <b>once</b> only<h3>

In [6]:
#import bselib.bse as bsl
import numpy as np
import pandas as pd
import requests
import json
import time
#from nsetools import Nse
import tkinter as tk
from tkinter import ttk
import pickle as pk
#nse = Nse()
#import os
#os.chdir('D:\my_jupyter_output')
#bl = bsl.BSE()
ticker_df = pd.read_csv('https://raw.githubusercontent.com/hurratanvir/algo-trade/master/Equity.csv?token=AOUGP5AQCC7RSHBJGIQDNLK67FQTO')
api_call_counter = 0
ticker_df.head(1)

,Security Code,Security Id,Security Name,Group,Face Value,ISIN No,Industry
0,500002,ABB,ABB India Limited,B,2.0,INE117A01022,Heavy Electrical Equipment


In [53]:
#API - Hurratanvir - BZHR969I14TKHTOV
#API - tanvirhurra - ETKZ6NPINJJXZ5MM
#API - tanveer_bmba2k19 - RW7OL5Y3XB452GYC
#API - rediffmail - LC02KQHE08R3EC52

In [2]:
def select_api():
    global api_call_counter
    api = ['BZHR969I14TKHTOV','ETKZ6NPINJJXZ5MM','RW7OL5Y3XB452GYC','LC02KQHE08R3EC52']
    key = api[api_call_counter]
    api_call_counter += 1
    if(api_call_counter >= len(api)):
        api_call_counter = 0
    return key

In [4]:
def populate_tab_portfolio(parent, child):
    parent.add(child, text ='Portfolio') 
    top_desc = ttk.Label(child, text = 'Total Companies in the Portfolio:')
    top_desc.grid(row =0, column = 0)
    top_desc_n = ttk.Label(child, text = '55')
    top_desc_n.grid(row =0, column = 2)
    try:
        top_desc_n['text'] = str(len(read_file(f_name)))
    except:
        top_desc_n['text'] = 0
    
    counter = 1
    try:
        data = read_file(f_name)
        #print(data)
        for i in data:
            ttk.Label(child, text = i).grid(row = counter, column = 0, sticky = 'W')
            ttk.Label(child, text = data[i]).grid(row = counter, column = 1, sticky = 'W')
            i = ttk.Button(child, text = 'Remove', command = lambda i=i: rem_comp(parent, child, i)).grid(row = counter, column = 2, sticky = 'E')
            counter+=1
    except:
        #c_ticker['text'] = 'You have nothing added yet'
        pass
    #container = ttk.Frame(root)
    canvas = tk.Canvas(child)
    scrollbar = ttk.Scrollbar(canvas, orient="vertical", command=canvas.yview)
    e = ttk.Entry(child)
    e.grid(column = 0, row = counter, ipadx = 30, columnspan = 2, pady =10, padx = 2)
    searchButton =ttk.Button(child, text = 'Search', command = lambda:callbac(e, c_ticker, add_company, counter))
    searchButton.grid(row = counter, column = 2, pady =10)
    c_ticker = ttk.Label(child)
    counter+=1
    c_ticker.grid(row =counter, column = 0, ipadx = 30, columnspan = 2,padx = 2)
    add_company = ttk.Button(child, text = 'Add Company to the Portfolio', command = lambda:add_comp_to_port(parent, child,e, c_ticker, add_company))
def rem_comp(parent, child, event):
    y = read_file(f_name)
    del y[event]
    save_file(f_name, y)
    for widget in child.winfo_children():
        widget.destroy()
    populate_tab_portfolio(parent, child)
    
    
def callbac(e, lbl, add_company, counter):
    c_name = ticker_df[ticker_df['Security Id'] == e.get().upper()]['Security Name']
    if(len(c_name)==0):
        c_name = 'Nothing Found!!!'
        add_company.grid_forget()
    else:
        c_name = c_name.iloc[0]
        add_company.grid(row = counter, column = 4)
    lbl['text'] = c_name
    
def add_comp_to_port(parent, child, e, c_ticker, add_company):
    tick = ticker_df[ticker_df['Security Id'] == e.get().upper()]['Security Id'].iloc[0]
    s_id = ticker_df[ticker_df['Security Id'] == e.get().upper()]['Security Code'].iloc[0]
    try:
        y = read_file(f_name)
        if(s_id in y):
            c_ticker['text'] = 'Company already exists in the portfolio'
            add_company.grid_forget()
        else:
            y[s_id] = tick
            save_file(f_name, y)
            c_ticker['text'] = 'Company added to the Portfolio'
            for widget in child.winfo_children():
                widget.destroy()
            populate_tab_portfolio(parent, child)
            #add_company.grid_forget()
            
    except:
        y = {}
        y[s_id] = tick
        save_file(f_name, y)
        for widget in child.winfo_children():
                widget.destroy()
        populate_tab_portfolio(parent, child)

In [5]:
root = tk.Tk()
tabControl = ttk.Notebook(root)
tab_portfolio = ttk.Frame(tabControl) 
tab_pead = ttk.Frame(tabControl) 

tabControl.add(tab_portfolio, text ='Portfolio') 
tabControl.add(tab_pead, text ='PEAD') 
tabControl.pack(expand = 1, fill ="both") 
populate_tab_portfolio(tabControl, tab_portfolio) 
#ttk.Label(tab1,  text ="Welcome to \ GeeksForGeeks").grid(column = 0,  row = 0, padx = 30, pady = 30)   
ttk.Label(tab_pead, text ="Lets dive into the\ world of computers").grid(column = 0, row = 0,  padx = 30, pady = 30) 
tabControl.config(width = 760, height = 500)
root.mainloop()

In [55]:
def get_me_stat(company_number, which_stat = "yoy_results"):
    fin = bl.historical_stats(company_number, stats=which_stat)
    if(len(fin) <=1 ):
        return 'search_error'
    mydf = pd.DataFrame(columns = list(fin.keys()))
    for i in range(len(list(fin.keys()))):
        mydf[mydf.columns[i]] = list(fin[list(fin.keys())[i]].values())
    mydf.set_index(mydf.columns[0], inplace = True)
    return mydf

In [56]:
def get_daily_timeseries(company):
   # time.sleep(5)
    api = select_api()
    company = company + '.bse'
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+company+'&apikey=' + api
    r = requests.get(url).json()
    try:
        rr = r[list(r.keys())[1]]
    except:
        print(r)
    my_df = pd.DataFrame()
    for x in rr.keys():
        my_df = my_df.append(rr[x], ignore_index = True)
    my_df.columns = ['open','high','low','close','volume']
    my_df.insert(0,'Date',rr.keys())
    my_df.set_index('Date', inplace = True)
    my_df = my_df.astype('float')
    return my_df

In [57]:
 def get_return(stock_data):
    stock_data['avg'] = stock_data[stock_data.columns[0:4]].sum(axis = 1)/4
    stock_data['return'] = (stock_data['avg'] - stock_data['avg'].shift(-1))/stock_data['avg'].shift(-1)
    return stock_data

In [58]:
def get_momentum(company):
    df = get_daily_timeseries(company)
    get_return(df)
    #print('completed')
    return (df[0:90]['return']+1).product() - 1

In [59]:
def get_me_sue(comp_number):
    income_df = get_me_stat(comp_number, "yoy_results")
    bs_df = get_me_stat(comp_number, "balancesheet")
    cf_df = get_me_stat(comp_number, "cashflow")
    if(isinstance(income_df,str) | isinstance(bs_df,str) or isinstance(cf_df,str)):
        return None
    #df.drop('TTM', inplace = True, axis = 1)
    x = [float(i) for i in income_df.iloc[-2].values]
    y = x[-1] - np.mean(x[-5:-1])
    y = y/np.std(x[-5:-1])
    return y

x = b.getScripCodes()
bl.comp_profile('523395')['symbol'].split('.')[0]

In [60]:
#dd = bl.historical_stats(590116, 'balancesheet')

In [61]:
#c_df = pd.read_csv('Equity.csv')
#s_df = pd.read_csv('sensex.csv')

count = 0
for i in range(len(s_df)):
    nse_p = float(nse.get_quote(s_df['Ticker'][i])['closePrice'])
    bse_p = float(bl.quote(s_df['Security Code'][i])['stockPrice'])
    arb = abs(nse_p-bse_p)
    if(arb >= 1):
        print('Arbitrage found')
        print('Comapny is '+s_df['COMPANY'][i]+' and the diff is '+ str(arb))
    count += 1
    print('completed '+ str(count)+' companies')

In [3]:
f_name = 'portfolio_list.pickle'
def read_file(file_name):
    pickle_in = open(file_name, 'rb')
    data = pk.load(pickle_in)
    pickle_in.close()
    return data

def save_file(file_name, data):
    pickle_out = open(file_name, 'wb')
    pk.dump(data, pickle_out)
    pickle_out.close()

In [65]:
x = {'a':'tanvir','b':'sajad'}

In [66]:
#conda update jupyter